# Earthquake Data Analysis

### Description

The catalog includes the magnitude, time of occurrence (s), and 3D coordinates (m) of earthquakes in about 20 years of recording in South California. Coordinates were converted from latitude, longitude, and depth of events in a seismic catalog. Magnitudes should be within the range $[0,8]$.

* **Waiting time (t)**: time interval between an event and the next one in the sequence.
* **Distance (r)**: Eucledian 3D distance between events. (each 3D set of coordinates refers to the hypocenter, i.e. the point triggering the slip in a fault that forms the earthquake)


### Assignments

1. Deduce what is the variable in each column of the catalog.
2. Visualize the process in space and/or time with suitable time series and/or 3D visualizations of the hypocenters. For instance, plot a space variable (a single coordinate or a nice linear combination of coordinates) as a function of time.
3. Compute the distribution $P_m(t)$ of waiting times for events of magnitude m or above (i.e. do not consider events below $m$). In shaping the bin sizes, take into account that this distribution is expected to have a power-law decay with time (e.g $\sim 1/t$), and that a power-law is well visualized in log-log scale. Do this analysis for many values of $m$, say $m=2,3,4,5$.
4. Compute the distribution $P_m(r)$ of the distance between an event and the next one, considering earthquakes of magnitude m or above. Also here make a clever choice for the bin sizes and try several values of $m$.
5. Compute the distribution $P_{m,R}(t)$ of waiting times for events of magnitude $m$ or above, which are separated by at most a distance $r<R$, for different values of m and $R$. (In this statistics, if the following event is farther than $R$, skip the $t$ and go to the next pair)
6. Eventually note if, from the analysis of the previous points, there emerges a scaling picture. Is there a suitable rescaling that collapses distributions for various $m$ (and eventually $R$ if point 5 is considered) on a single curve?

### Datasets

* column 1: index of the event
* column 2: index of the previous event that triggered it (defined with a given algorithm), -1 if no ancestor is found
* column 3: time (seconds) from 0:00 of Jan.1st, 1982
* column 4: magnitude
* columns 5, 6, and 7: 3D coordinates (meters) of the earthquake hypocenter, i.e. of the point from where it started. These Euclidean coordinates are derived from latitude, longitude and depth.

Joining each event to that with the index of the second column (if not -1), there emerges a set of causal trees.


### Contact
* Marco Baiesi <marco.baiesi@unipd.it>

# 1. Data Acquisition

We load the given dataset, organize it in a pandas DataFrame and assign a name to each column.

In [ ]:
# google colab upload link, to be omitted when working on Jupyter Notebook
from google.colab import drive
drive.mount('/content/drive')
filepath = '/content/drive/MyDrive/LCP_project/'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import plotly.express as px
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns
import numpy.linalg as la

In [ ]:
data = 'SouthCalifornia-1982-2011_Physics-of-Data.dat'

In [ ]:
# google colab command
df = pd.read_csv(data, sep = ' ', names = ['Index', 'Prev_event', 'Time', 'Magnitude', 'x', 'y', 'z'])

# Jupyter Notebook command
# df = pd.read_csv(data, sep = ' ', names = ['Index', 'Prev_event', 'Time', 'Magnitude', 'x', 'y', 'z'])
df

In [ ]:
# AGGIUNTA COLONNE PER I DATI AGGIUNTIVI: Waiting time, Distance, lat, lon, dep
df['Waiting time prev'] = df['Index']
df['Distances prev'] = df['Index']
df['Waiting time'] = df['Index']
df['Distances'] = df['Index']

for i in range(np.shape(df)[0]):
  if df.loc[i, 'Prev_event'] == -1:
    Time_1 = df.loc[i, 'Time']
    Distance_1 = np.array([df['x'][i], df['y'][i], df['z'][i]])
  elif df.loc[i, 'Prev_event'] > -1:
    Time_1 = df['Time'][df['Prev_event'][i]]
    Distance_1 = np.array([df['x'][df['Prev_event'][i]], df['y'][df['Prev_event'][i]], df['z'][df['Prev_event'][i]]])

  df.loc[i, 'Waiting time prev'] = df['Time'][i] - Time_1
  df.loc[i, 'Distances prev'] = math.sqrt(np.sum((np.array([df['x'][i], df['y'][i], df['z'][i]]) - Distance_1)**2))


for i in range(np.shape(df)[0]):
  if df['Index'][i] == 0:
    Time_1 = df.loc[i, 'Time']
    Distance_1 = np.array([df.loc[i, 'x'], df.loc[i, 'y'], df.loc[i, 'z']])
  elif df.loc[i, 'Index'] > -0:
    Time_1 = df.loc[i-1, 'Time']
    Distance_1 = np.array([df.loc[i-1, 'x'], df.loc[i-1, 'y'], df.loc[i-1, 'z']])

  df.loc[i, 'Waiting time'] = df.loc[i, 'Time'] - Time_1
  df.loc[i, 'Distances'] = math.sqrt(np.sum((np.array([df.loc[i, 'x'], df.loc[i, 'y'], df.loc[i, 'z']]) - Distance_1)**2))

R = 6371000 # m
df['r'] = df['Index']
for i in range(np.shape(df)[0]):
  df.loc[i, 'r'] = (df.loc[i, 'x']**2 + df.loc[i, 'y']**2 + df.loc[i, 'z']**2)**0.5

df['lat'] = np.arcsin(df['z'] / df['r'])*180/math.pi
df['lon'] = np.arctan2(df['y'], df['x'])*180/math.pi
df['dep'] = df['r'] - R

df

# 2. Graphical Visualization

## 2.1 Plots of the geographical coordinates

In the followig section we present some 2D and 3D plots of the earthquakes' hypocenter locations with respect to the time and magnitude variables.

### 2.1.1 Static maps

In [ ]:
fig = plt.figure(figsize=(16,9))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=df['dep'], cmap='inferno_r', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='depth', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=df['dep'], cmap='inferno_r', marker='.')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='depth', shrink=0.7)

ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df['x'], df['y'], df['z'], c=df['dep'], cmap='inferno_r', marker='.')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,150)
fig.colorbar(scatter3, label='depth', shrink=0.5)

ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df['lon'], df['lat'], df['dep'], c=df['dep'], cmap='inferno_r', marker='.')
ax4.set_xlabel('longitude')
ax4.set_ylabel('latitude')
ax4.set_zlabel('depth')
ax4.view_init(20,90)
fig.colorbar(scatter4, label='depth', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df['lon'], df['lat'], c=df['dep'], cmap='inferno_r', marker='.')
ax5.set_xlabel('longitude')
ax5.set_ylabel('latitude')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='depth', shrink=0.7)

df_mag4 = df[df['Magnitude'] > 4]
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df_mag4['lon'], df_mag4['lat'], df_mag4['dep'], c=df_mag4['dep'], cmap='inferno_r', marker='.')
ax6.set_xlabel('longitude')
ax6.set_ylabel('latitude')
ax6.set_zlabel('depth')
ax6.set_title('Magnitude > 4')
ax6.view_init(20,90)
fig.colorbar(scatter6, label='depth', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,9))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='Waiting time log', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Waiting time log', shrink=0.7)

ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df['x'], df['y'], df['z'], c=df['Time'], cmap='viridis', marker='.')
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,30)
fig.colorbar(scatter3, label='Time', shrink=0.5)

ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df['lon'], df['lat'], df['dep'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax4.set_xlabel('longtude')
ax4.set_ylabel('latitude')
ax4.set_zlabel('depth')
ax4.view_init(20,90)
fig.colorbar(scatter4, label='Waiting time log', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df['lon'], df['lat'], c=np.log(df['Waiting time']), cmap='viridis', marker='.')
ax5.set_xlabel('longitude')
ax5.set_ylabel('latitude')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='Waiting time log', shrink=0.7)

ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df['lon'], df['lat'], df['dep'], c=df['Time'], cmap='viridis', marker='.')
ax6.set_xlabel('longitude')
ax6.set_ylabel('latitude')
ax6.set_zlabel('depth')
ax6.view_init(20,90)
fig.colorbar(scatter6, label='Time', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,9))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['x'], df['y'], df['z'], c=df['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)
fig.colorbar(scatter1, label='Magnitude', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['x'], df['y'], c=np.log(df['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Magnitude log', shrink=0.7)

df_mag3 = df[df['Magnitude'] > 3]
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df_mag3['x'], df_mag3['y'], df_mag3['z'], c=np.log(df_mag3['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('z')
ax3.view_init(0,30)
fig.colorbar(scatter3, label='Magnitude log > 3', shrink=0.5)

df_mag4 = df[df['Magnitude'] > 4]
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df_mag4['x'], df_mag4['y'], df_mag4['z'], c=df_mag4['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('z')
ax4.view_init(0,30)
fig.colorbar(scatter4, label='Magnitude > 4', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df_mag4['x'], df_mag4['y'], c=np.log(df_mag4['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax5.set_xlabel('x')
ax5.set_ylabel('y')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='Magnitude log > 4', shrink=0.7)

df_mag6 = df[df['Magnitude'] > 6]
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df_mag6['x'], df_mag6['y'], df_mag6['z'], c=df_mag6['Magnitude'], cmap='plasma', marker='o', alpha=0.8, vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax6.set_xlabel('x')
ax6.set_ylabel('y')
ax6.set_zlabel('z')
ax6.view_init(0,30)
fig.colorbar(scatter6, label='Magnitude log > 6', shrink=0.5)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,10))

ax1 = fig.add_subplot(2, 3, 1, projection='3d')
scatter1 = ax1.scatter(df['lon'], df['lat'], df['dep'], c=df['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax1.set_xlabel('longitude')
ax1.set_ylabel('latitude')
ax1.set_zlabel('depth')
ax1.view_init(20,90)
fig.colorbar(scatter1, label='Magnitude', shrink=0.5)

ax2 = fig.add_subplot(2, 3, 2)
scatter2 = ax2.scatter(df['lon'], df['lat'], c=np.log(df['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_aspect(aspect=1)
fig.colorbar(scatter2, label='Magnitude log', shrink=0.7)

df_mag3 = df[df['Magnitude'] > 3]
ax3 = fig.add_subplot(2, 3, 3, projection='3d')
scatter3 = ax3.scatter(df_mag3['lon'], df_mag3['lat'], df_mag3['dep'], c=np.log(df_mag3['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax3.set_xlabel('longitude')
ax3.set_ylabel('latitude')
ax3.set_zlabel('depth')
ax3.view_init(20,90)
fig.colorbar(scatter3, label='Magnitude log > 3', shrink=0.5)

df_mag4 = df[df['Magnitude'] > 4]
ax4 = fig.add_subplot(2, 3, 4, projection='3d')
scatter4 = ax4.scatter(df_mag4['lat'], df_mag4['lon'], df_mag4['dep'], c=df_mag4['Magnitude'], cmap='plasma', marker='.', vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax4.set_xlabel('longitude')
ax4.set_ylabel('latitude')
ax4.set_zlabel('depth')
ax4.view_init(20,90)
fig.colorbar(scatter4, label='Magnitude > 4', shrink=0.5)

ax5 = fig.add_subplot(2, 3, 5)
scatter5 = ax5.scatter(df_mag4['lon'], df_mag4['lat'], c=np.log(df_mag4['Magnitude']), cmap='plasma', marker='.', vmin=np.min(np.log(df.Magnitude)), vmax=np.max(np.log(df.Magnitude)))
ax5.set_xlabel('longitude')
ax5.set_ylabel('latitude')
ax5.set_aspect(aspect=1)
fig.colorbar(scatter5, label='Magnitude log > 4', shrink=0.7)

df_mag6 = df[df['Magnitude'] > 6]
ax6 = fig.add_subplot(2, 3, 6, projection='3d')
scatter6 = ax6.scatter(df_mag6['lon'], df_mag6['lat'], df_mag6['dep'], c=df_mag6['Magnitude'], cmap='plasma', marker='o', alpha=0.8, vmin=np.min(df.Magnitude), vmax=np.max(df.Magnitude))
ax6.set_xlabel('longitude')
ax6.set_ylabel('latitude')
ax6.set_zlabel('depth')
ax6.view_init(20,90)
fig.colorbar(scatter6, label='Magnitude log > 6', shrink=0.5)

plt.tight_layout()

### 2.1.2 Tree structure visualization (a)

In [ ]:
df_0 = df[df['Prev_event'] > 0]
df_1 = df[df['Prev_event'] == 0]
print(np.shape(df_0))

fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1, 2, 1, projection='3d')
scatter1_1 = ax1.scatter(df_0['x'], df_0['y'], df_0['z'], c=df_0['Prev_event'], cmap='rainbow', marker='.')
scatter1_2 = ax1.scatter(df_1['x'], df_1['y'], df_1['z'], c='black', marker='.')
fig.colorbar(scatter1_1, label='Prev_event', shrink=0.5)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')
ax1.view_init(0,30)

ax2= fig.add_subplot(1, 2, 2, projection='3d')
scatter2_1 = ax2.scatter(df_0['lon'], df_0['lat'], df_0['dep'], c=df_0['Prev_event'], cmap='rainbow', marker='.')
scatter2_2 = ax2.scatter(df_1['lon'], df_1['lat'], df_1['dep'], c='black', marker='.')
fig.colorbar(scatter2_1, label='Prev_event', shrink=0.5)
ax2.set_xlabel('longitude')
ax2.set_ylabel('latitude')
ax2.set_zlabel('depth')
ax2.view_init(20,90)

plt.tight_layout()

### 2.1.3 Interactive and animated maps

In [ ]:
ax1 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.Magnitude, width=600, height=500, opacity=0.5, color_continuous_scale='plasma',
                        center=dict(lat=32, lon=-117), zoom=4, title='Interactive map of the South California earthquakes',
                        mapbox_style='open-street-map')
ax1.show()

ax2 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.dep, width=600, height=500, opacity=0.5, color_continuous_scale='inferno_r',
                        center=dict(lat=32, lon=-117), zoom=4, title='Interactive map of the South California earthquakes',
                        mapbox_style='open-street-map')
ax2.show()

In [ ]:
df['Animation_frame'] = np.round(200*(df['Time']/df['Time'].max()))

In [ ]:
ax1 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.Magnitude, width=600, height=500, opacity=1, color_continuous_scale='plasma', range_color=[df.Magnitude.min(), df.Magnitude.max()],
                        center=dict(lat=33.5, lon=-117), zoom=4, title='Interactive map of the South California earthquakes', animation_frame=df.Animation_frame,
                        mapbox_style='open-street-map')
ax1.show()

ax2 = px.scatter_mapbox(df, lat=df.lat, lon=df.lon, color=df.dep, width=600, height=500, opacity=1, color_continuous_scale='inferno_r', range_color=[df.dep.min(), df.dep.max()],
                        center=dict(lat=33.5, lon=-117), zoom=4, title='Interactive map of the South California earthquakes', animation_frame=df.Animation_frame,
                        mapbox_style='open-street-map')
ax2.show()

### 2.1.4 Distribution plots

In [ ]:
sns.jointplot(df, x=df.lon, y=df.lat, kind='hist')

In [ ]:
sns.kdeplot(df, x=df.lon, y=df.lat, fill=True, thresh=0, levels=15, cmap='rocket_r')

In [ ]:
sns.jointplot(df, x=df.dep, y=df.Magnitude, kind='hist')

### 2.1.5 Time analysis

In [ ]:
fig = plt.figure(figsize=(15,4))

ax1 = fig.add_subplot(1,3,1)
ax1.plot(df.Time, df.Index, label='Time vs Index')
ax1.set_xlabel('Time')
ax1.set_ylabel('Index')

i=100000
ax2 = fig.add_subplot(1,3,2)
ax2.scatter(df.Time[i:i+1000], df.Magnitude[i:i+1000], marker='.')
ax2.set_xlabel('Time')
ax2.set_ylabel('Magnitude')

ax3 = fig.add_subplot(1,3,3)
ax3.scatter(df.Magnitude[i:i+1000], df.dep[i:i+1000], marker='.')
ax3.set_xlabel('Magnitude')
ax3.set_ylabel('Depth')

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(13,3.5))
ax = fig.add_subplot(1,1,1)

n, bins, _ = ax.hist(df.Time, bins=5*int(np.sqrt(np.shape(df)[0])))

ax.set_xticks([i for i in 0.5e8*np.arange(20)])
ax.set_xlim(0, np.max(df.Time))
ax.set_xlabel('Time (10^8 s)')
ax.set_ylabel('Counts')

In [ ]:
centroids = bins[:-1] - bins[1:]
event_time_dist = np.concatenate((centroids.reshape(np.shape(centroids)[0], 1), n.reshape(np.shape(n)[0], 1)), axis=1)

from scipy.fftpack import fft, ifft
spectrum = abs(fft(n))

fig = plt.figure(figsize=(13,3.5))
ax = fig.add_subplot(1,1,1)
ax.scatter(np.arange(1660), spectrum, marker='.')

np.shape(spectrum)

In [ ]:
fig = plt.figure(figsize=(13,3.5))
ax = fig.add_subplot(1,1,1)

n, bins, _ = ax.hist(df['Waiting time'], bins=5*int(np.sqrt(np.shape(df)[0])))

ax.set_xticks([i for i in 10000*np.arange(11)])
ax.set_xlim(0, 1e5)
ax.set_xlabel('Waiting time (s)')
ax.set_ylabel('Counts')

## 2.2 Principal Component Analysis

In [ ]:
PCA_array = np.array(df)[:, 4:7]
print(np.shape(PCA_array))

C = np.cov(PCA_array.T)
sns.heatmap(C)

In [ ]:
U, spectrum, Vt = la.svd(PCA_array[:10000].T)

In [ ]:
l_svd = spectrum**2/(np.shape(df)[0]+1)
V_svd = U

print('Autovalori\t', l_svd/C.trace())
print('Autovettore\t', V_svd[:,0])

In [ ]:
array_newbasis = np.array([la.solve(U, PCA_array[i, :]) for i in range(np.shape(PCA_array)[0])])

In [ ]:
np.shape(array_newbasis)

In [ ]:
fig = plt.figure(figsize=(16,5))

ax1 = fig.add_subplot(1,2,1, projection='3d')

scat1 = ax1.scatter(array_newbasis[:,0], array_newbasis[:,1], array_newbasis[:,2], marker='.', c=array_newbasis[:,0], cmap='inferno')
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_zlabel('PC3')
ax1.set_xlim(np.mean(array_newbasis[:,0])-1000000/2, np.mean(array_newbasis[:,0])+1000000/2)
ax1.set_aspect(aspect='equal')
fig.colorbar(scat1, label='PC1', shrink=0.5)

ax2 = fig.add_subplot(1,2,2, projection='3d')

scat2 = ax2.scatter(array_newbasis[:,0], array_newbasis[:,1], array_newbasis[:,2], marker='.', c=array_newbasis[:,0], cmap='inferno')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC2')
ax2.set_zlabel('PC3')
ax2.set_xlim(np.mean(array_newbasis[:,0])-1000000/2, np.mean(array_newbasis[:,0])+1000000/2)
ax2.set_aspect(aspect='equal')
ax2.view_init(0,90)
fig.colorbar(scat2, label='PC1', shrink=0.5)


# 3. Distribution $P_m(t)$ of the Waiting Times

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from astropy.stats import knuth_bin_width as knuth
from scipy.stats import chi2
from scipy.optimize import curve_fit

First we define the functions we need for data analysis and plots.

The `log_plot` function sets up all parameters we need for plotting histogram in log-log scale:

* To select number of bins, we use used the Knuth binning rule (*$``\text{Optimal Data-Based Binning for Histograms}"$, Kevin H. Knuth, 2013*).

* We generate an equally logarithmically spaced set of points using `np.logspace()` specifying the number of bins we calculated before.

* After evaluating bin centers and bin widths we normalize the histogram in order to give a statistical meaning to the distribution.

* We get rid of the empty bins in order for us to be able to evaluate the errors propagating the Poisson error of the unnormalized histogram.

* The function returns the bin centers, the counts per bin and the errors (all in logarithmic scale).

In [ ]:
def log_plot(feature):

    #Excluding zeros
    feature = feature[feature != 0]

    #Discard consecutive elements of the feature to avoid correlations
    feature = feature[::2]

    #Bin edges (in logarithmic scale)
    #n_bins = max([np.histogram_bin_edges(np.log10(feature), bins = 'scott').shape[0] - 1, 10])
    _, bin_edges = knuth(np.log10(feature), return_bins = True)

    x = np.logspace(np.log10(feature.min()), np.log10(feature.max()), base = 10, num = bin_edges.shape[0] - 1)

    #Histogram data
    hist = np.histogram(feature, bins = x)
    bin_centers = (hist[1][1:] + hist[1][:-1]) / 2
    bin_widths = hist[1][1:] - hist[1][:-1]

    #Normalization of the histogram
    norm = np.sum(hist[0])
    counts_normalized = hist[0] / (bin_widths * norm)

    #Clean of empty bins
    mask = (counts_normalized > 0)
    counts_normalized = counts_normalized[mask]
    bin_centers = bin_centers[mask]

    bins = np.log10(bin_centers)
    counts = np.log10(counts_normalized)
    errors = np.log10(np.e) / np.sqrt(hist[0][mask])

    return bins, counts, errors

As discussed later, we use `cut_tails` function to proper select the linearly distributed points (in log-log scale):

* For the left side we first arbitraly drop off all data points a given threshold

* Then, for both sides we recursively compare the pearson coefficient with the one we would obtain getting rid off of the extreme point: if the new value is better than the previous one we reject that point, otherwise we stop the algorithm.

* The function returns bin centers, counts per bin, errors and the pearson coefficients list.

In [ ]:
def cut_tails(b, c, e, threshold):

    c = c[b >= threshold]
    e = e[b >= threshold]
    b = b[b >= threshold]

    pearson = [0]

    while True:
        new_p = np.corrcoef(b[:-1], y = c[:-1])[0,1]
        if(new_p > pearson[-1] or len(b) <= 9):
            break
        b = b[:-1]
        c = c[:-1]
        e = e[:-1]
        pearson.append(new_p)

    while True:
        new_p = np.corrcoef(b[1:], y = c[1:])[0,1]
        if(new_p > pearson[-1] or len(b) <= 9):
            break
        b = b[1:]
        c = c[1:]
        e = e[1:]
        pearson.append(new_p)

    return b, c, e, pearson

The `statistic_plots` function allows us to perform a linear regression on the given feature and to evaluate the $\chi^2$.

In [ ]:
f_lin = lambda x, m, q: m*x+q

def statistic_plots(bins, counts, magnitudes, errors, feature, spec = '', show_plots=True):
    fit_res = []

    if(show_plots):
        fig, ax = plt.subplots(2, len(magnitudes), figsize = (14, 6))

    for i, (x, y, m, e) in enumerate(zip(bins, counts, magnitudes, errors)):

        #Linear regression
        (slope, intercept), pcov = curve_fit(f_lin, x, y, sigma=e, absolute_sigma=True)
        
        #Residuals
        residuals = y - (slope*x+intercept)
        chisquare = np.sum((residuals/e)**2)
        dof = x.shape[0]-2 #degrees of freedom of the chisquare
        fit_res.append((slope, intercept, pcov, chisquare, dof))

        #Plot
        if(show_plots):
            x_axis = np.linspace(x.min(), x.max(), 100)
            ax[0,i].scatter(x, y, label="Data points")
            ax[0,i].plot(x_axis, (slope * x_axis) + intercept, color = 'darkred', label = 'Fit')
            ax[0,i].set_ylabel(f"$\log(P_{m}({feature})$)")
            ax[0,i].set_xlabel(f"$\log({feature})$")
            ax[0,i].grid()

            #Residual plot
            ax[1,i].errorbar(x, residuals, yerr = e, fmt = 'o', capsize = 2.5, label="Residual")
            ax[1,i].hlines(0,x.min(), x.max(), color='darkred', label = 'Fit')
            ax[1,i].grid()

    if(show_plots):
        ax[0,-1].legend()
        ax[1,-1].legend()
        fig.suptitle('Linear regressions and residuals' + spec, fontsize = 18)
        plt.tight_layout()
        plt.show()
    
    return fit_res

As features we need waiting times, that we can get by applying the `diff()` function to the 'Time' column of dataframe starting from the second row.

In [ ]:
#Inizializing lists
magnitudes = [2, 3, 4, 5]
bins = []
counts = []
errors = []

for i, m in enumerate(magnitudes):

    #Array of waiting times
    wt = np.array(df[df['Magnitude'] >= m][['Time']].sort_values('Time').diff())[1:]

    #Prepare data for plot
    b, c, e = log_plot(wt)
    bins.append(b)
    counts.append(c)
    errors.append(e)

    #Plot
    plt.scatter(10 ** b, 10 ** c, alpha = 0.75, label = f"m $\geq$ {m}")
    plt.xscale('log', base = 10)
    plt.yscale('log', base = 10)

plt.xlabel('Waiting times ($s$)')
plt.ylabel('Probability density ($Hz$)')
plt.title('Distribution of waiting times for different magnitudes')
plt.legend()
plt.grid()
plt.show()

We expect a power-law distribution of the waiting times for all magnitudes of type

$$P_m(wt)=kwt^A$$

Performing a logarithmic transformation on the data that gives

$$\log_{10}P_m(wt)=A\log_{10}(wt)+B$$

i.e. a linear distribution.

So, in order to prove this, we will apply a linear regression and verify this is the law followed by data.

Now we can apply the `cut_tails()` function in order to select only the linear regime of the distributions.

In [ ]:
pearson_list = []
last_cut_list = []

counts_cut = counts[:]
errors_cut = errors[:]
bins_cut = bins[:]

for i, m in enumerate(magnitudes):
    
    bins_cut[i], counts_cut[i], errors_cut[i], pearson = cut_tails(bins[i], counts[i], errors[i], threshold = 2)
    pearson_list.append(pearson[-1])
    last_cut_list.append(bins_cut[i][-1])

Finally, we perform the linear regression and visualize both the fit and residuals.

In [ ]:
fit_res = statistic_plots(bins_cut, counts_cut, magnitudes, errors_cut, 'wt')

We display the results of the linear fit.

In [ ]:
slopes = []
intercepts = []
sigmas_slope = []
sigmas_intercept = []

for res, m, p in zip(fit_res, magnitudes, pearson_list):
    slope, intercept, pcov, chisquare , dof = res
    confidence_level = (1.- chi2.cdf(chisquare, dof))*100
    
    slopes.append(slope)
    intercepts.append(intercept)

    sigma_slope, sigma_intercept = np.sqrt(np.diag(pcov))
    
    sigmas_slope.append(sigma_slope)
    sigmas_intercept.append(sigma_intercept)

    print(f"------ Earthquakes with magnitude m ≥ {m}  ------")
    print(f"Slope: {slope:.3f} ± {sigma_slope:.3f}")
    print(f"Intercept: {intercept:.3f} ± {sigma_intercept:.3f}")
    print(f"Pearson coefficient: {p:.2f}")
    print(f"Degrees of freedom: {dof}")
    #print(f"Chi2: {chisquare:.2f}")
    #print(f"Hypotesis accepted with confidence level {confidence_level:.1f}%")
    print()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (16,6))

ax[0].scatter(magnitudes, last_cut_list)
ax[0].set_xlabel('Magnitudes')
ax[0].set_ylabel('$wt_{max}$')
ax[0].grid()

ax[1].errorbar(magnitudes, slopes, sigmas_slope)
ax[1].set_xlabel('Magnitudes')
ax[1].set_ylabel('Slopes')
ax[1].grid()

ax[2].errorbar(magnitudes, intercepts, sigmas_intercept)
ax[2].set_xlabel('Magnitudes')
ax[2].set_ylabel('Intercepts')
ax[2].grid()

plt.tight_layout()
plt.show()

We notice two relevant things:

* Slopes and intercepts of linear regressions seems to be close to the same value: $-1$ for the former and $-1.5$ for the latter; we will later investigate whether these parameters are universal constants or they depends on the choice of magnitudes and maximum distances.

* Let's define $wt_{max}$ as the maximum value of waiting time in linear regime. We can observe that, after this threshold the distribution drops significantly. Also, $wt_{max}$ increases with the growth of magnitude.

In order to collapse each distribution to a single trending line we produce a new plot of the probability density function normalized with respect to the regressions we obtained before versus the waiting times normalized by $wt_{max}$.

In [ ]:
for b, c, e, m, last_cut, slope, intercept in zip(bins, counts, errors, magnitudes, last_cut_list, slopes, intercepts):
    
    c = c[b >= last_cut-2.5]
    e = e[b >= last_cut-2.5]
    b = b[b >= last_cut-2.5] 
        
    plt.plot((10 ** b) / (10 ** last_cut), (10 ** c) / (10 ** (slope * b + intercept)), alpha = 1, linestyle = '--', marker = 'o', label = f"m $\geq$ {m}")
    plt.xscale('log', base = 10)
    plt.yscale('log', base = 10)
    
plt.xlabel("$\\frac{wt}{wt_{max}}$")
plt.ylabel("$\\frac{P_m(wt)}{wt^A\cdot10^B}$")
plt.title("Scaling picture plot")
plt.grid()
plt.legend()
plt.show()

# 4. Distribution $P_m(r)$ of the Distances

We repeat what we did in the previous point for the distances between two consecutive measurement.

In [ ]:
magnitudes = [2.5, 3, 3.5, 4, 4.5]

bins   = []
counts = []
errors = []

for i, m in enumerate(magnitudes):
      #Array of siatcnes
      df2 = df[df['Magnitude'] >= m].sort_values('Time')[['x', 'y', 'z']]
      coordinates = df2.to_numpy()
      d = np.linalg.norm(np.diff(coordinates, axis = 0), axis = 1)

      #Prepare data for plot
      b, c, e = log_plot(d)
      bins.append(b)
      counts.append(c)
      errors.append(e)

      #Plot
      plt.scatter(10 ** b, 10 ** c, alpha = 0.75, label = f"m $\geq$ {m}")
      plt.xscale('log', base = 10)
      plt.yscale('log', base = 10)

plt.xlabel('Distances ($m$)')
plt.ylabel('Probability density ($m^{-1}$)')
plt.title('Distribution of distances for different magnitudes')
plt.legend()
plt.grid()
plt.show()

In [ ]:
pearson_list = []
last_cut_list = []

counts_cut = counts[:]
errors_cut = errors[:]
bins_cut = bins[:]

for i, m in enumerate(magnitudes):
    
    bins_cut[i], counts_cut[i], errors_cut[i], pearson = cut_tails(bins[i], counts[i], errors[i], threshold = 2)
    pearson_list.append(pearson[-1])
    last_cut_list.append(bins_cut[i][-1])

In [ ]:
fit_res = statistic_plots(bins_cut, counts_cut, magnitudes, errors_cut, 'd')

We display the results of the linear fit.

In [ ]:
slopes = []
intercepts = []
sigmas_slope = []
sigmas_intercept = []

for res, m, p in zip(fit_res, magnitudes, pearson_list):
    slope, intercept, pcov, chisquare , dof = res
    confidence_level = (1.- chi2.cdf(chisquare, dof))*100
    
    slopes.append(slope)
    intercepts.append(intercept)

    sigma_slope, sigma_intercept = np.sqrt(np.diag(pcov))
    
    sigmas_slope.append(sigma_slope)
    sigmas_intercept.append(sigma_intercept)

    print(f"------ Earthquakes with magnitude m ≥ {m}  ------")
    print(f"Slope: {slope:.3f} ± {sigma_slope:.3f}")
    print(f"Intercept: {intercept:.3f} ± {sigma_intercept:.3f}")
    print(f"Pearson coefficient: {p:.2f}")
    print(f"Degrees of freedom: {dof}")
    #print(f"Chi2: {chisquare:.2f}")
    #print(f"Hypotesis accepted with confidence level {confidence_level:.1f}%")
    print()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (16,6))

ax[0].scatter(magnitudes, last_cut_list)
ax[0].set_xlabel('Magnitudes')
ax[0].set_ylabel('$wt_{max}$')
ax[0].grid()

ax[1].errorbar(magnitudes, slopes, sigmas_slope)
ax[1].set_xlabel('Magnitudes')
ax[1].set_ylabel('Slopes')
ax[1].grid()

ax[2].errorbar(magnitudes, intercepts, sigmas_intercept)
ax[2].set_xlabel('Magnitudes')
ax[2].set_ylabel('Intercepts')
ax[2].grid()

plt.tight_layout()
plt.show()

In contrast to waiting times, it seems there isn't a clear correlation between magnitudes and $d_{max}$, so we will recreate the scaling picture plot without normalizing the x axis.

In [ ]:
for b, c, e, m, last_cut, slope, intercept in zip(bins, counts, errors, magnitudes, last_cut_list, slopes, intercepts):
    
    c = c[b >= last_cut-2.5]
    e = e[b >= last_cut-2.5]
    b = b[b >= last_cut-2.5] 
        
    plt.plot((10 ** b), (10 ** c) / (10 ** (slope * b + intercept)), alpha = 1, linestyle = '--', marker = 'o', label = f"m $\geq$ {m}")
    plt.xscale('log', base = 10)
    plt.yscale('log', base = 10)
    
plt.xlabel("$d \ (m)$")
plt.ylabel("$\\frac{P_m(d)}{d^A\cdot10^B}$")
plt.title("Scaling picture plot")
plt.grid()
plt.legend()
plt.show()

# 5. Distribution $P_{m,R}(t)$ of Waiting Times for events at Distance $r<R$

We construct a function  - called `get_Pmr()` - that, for a given value of magnitude $m$ and relative distance $R$, selects the events having magnitude $\geqslant m$ and relative distances $\leqslant R$; finally, it computes the waiting times.

In [ ]:
def get_Pmr(df, m, R):

    dfm = df[df['Magnitude']>= m].sort_values('Time')[['Time', 'x', 'y', 'z']]

    #Evaluate the distance between one event and the next one
    coordinates = dfm.to_numpy()
    d = np.linalg.norm(np.diff(coordinates, axis = 0), axis = 1)
    d = np.insert(d, 0, 0)

    dfm['Rel_distance'] = d
    dfmr = dfm[dfm['Rel_distance'] <= R].sort_values('Time') #Select by distance

    wt = np.array(dfmr['Time'].diff())[1:]

    return dfmr, d, wt

Now we can plot the results using two nested cycles: with the outer one we select the magnitude and the inner one the relative distance.

In contrast to the previous calculations we excluded the highest magnitude $m=5$ because the plot resulted into being too unpopulated.

In [ ]:
magnitudes = [2, 2.5, 3, 3.5, 4]
Radii = [1e4, 5e4, 1e5, 5e5, 1e6]

bins   = [[] for _ in range(len(Radii))]
counts = [[] for _ in range(len(Radii))]
errors = [[] for _ in range(len(Radii))]

for j, radius in enumerate(Radii):
    index = 0
    fig, ax = plt.subplots(1, len(magnitudes), figsize = (14,5))

    for i, m in enumerate(magnitudes):
        dfmr, d, wt = get_Pmr(df, m, radius)

        b, c, e = log_plot(wt)
        bins[j].append(b)
        counts[j].append(c)
        errors[j].append(e)

        ax[index].scatter(10 ** b, 10 ** c, edgecolor = 'black', marker = '.')
        ax[index].set_title(f'm $\geqslant$ {m} and R $\leqslant$ {radius:.0f}')
        ax[index].set_yscale('log', base = 10)
        ax[index].set_xscale('log', base = 10)

        ax[index].set_ylabel(f"$\log(P_{m}(wt)$)")
        ax[index].set_xlabel(f"$\log(wt)$")
        ax[index].grid()

        index += 1

    plt.tight_layout()

plt.show()

In [ ]:
pearson_list = []
last_cut_list = [[] for _ in range(len(Radii))]

counts_cut = counts[:]
errors_cut = errors[:]
bins_cut = bins[:]

for i, m in enumerate(magnitudes):
    for j, radius in enumerate(Radii):
    
        bins_cut[j][i], counts_cut[j][i], errors_cut[j][i], pearson = cut_tails(bins_cut[j][i], counts_cut[j][i], errors_cut[j][i], threshold = 2)
        pearson_list.append(pearson[-1])
        last_cut_list[j].append(bins_cut[j][i][-1])

In [ ]:
slopes = [[] for _ in range(len(Radii))]
intercepts = [[] for _ in range(len(Radii))]
sigmas_slope = [[] for _ in range(len(Radii))]
sigmas_intercept = [[] for _ in range(len(Radii))]

for j, radius in enumerate(Radii):

    fit_res = statistic_plots(bins_cut[j], counts_cut[j], magnitudes, errors_cut[j], 'wt', spec = f' ($R\leqslant{radius:.0f}$)')

    for res, m, p in zip(fit_res, magnitudes, pearson_list):
        slope, intercept, pcov, chisquare , dof = res
        
        slopes[j].append(slope)
        intercepts[j].append(intercept)

        confidence_level = (1.- chi2.cdf(chisquare, dof))*100

        sigma_slope, sigma_intercept = np.sqrt(np.diag(pcov))
        
        sigmas_slope[j].append(sigma_slope)
        sigmas_intercept[j].append(sigma_intercept)

        print(f"------ Earthquakes with m ≥ {m}, R ≤ {radius}  ------")
        print(f"Slope: {slope:.3f} ± {sigma_slope:.3f}")
        print(f"Intercept: {intercept:.3f} ± {sigma_intercept:.3f}")
        print(f"Pearson coefficient: {p:.2f}")
        print(f"Degrees of freedom: {dof}")
        #print(f"Chi2: {chisquare:.2f}")
        #print(f"Hypotesis accepted with confidence level {confidence_level:.1f}%")
        print()

In [ ]:
for i, r in enumerate(Radii):
    for b, c, e, m, last_cut, slope, intercept in zip(bins[i], counts[i], errors[i], magnitudes, last_cut_list[i], slopes[i], intercepts[i]):
        
        c = c[b >= last_cut-2.5]
        e = e[b >= last_cut-2.5]
        b = b[b >= last_cut-2.5] 
        
        plt.plot((10 ** b) / (10 ** last_cut), (10 ** c) / (10 ** (slope * b + intercept)), alpha = 1, linestyle = '--', marker = 'o', label = f"m $\geq$ {m}")
        
plt.xscale('log', base = 10)
plt.yscale('log', base = 10)    
plt.xlabel("$d \ (m)$")
plt.ylabel("$\\frac{P_m(d)}{d^A\cdot10^B}$")
plt.title("Scaling picture plot")
plt.grid()
plt.show()

In [ ]:
for i, r in enumerate(Radii):
    for j, (b, c, e, m, last_cut, slope, intercept) in enumerate(zip(bins[i], counts[i], errors[i], magnitudes[:-1], last_cut_list[i], slopes[i], intercepts[i])):
        if i % 2 == 1 or j % 2 == 1:
            continue
        
        c = c[b >= last_cut-2.5]
        e = e[b >= last_cut-2.5]
        b = b[b >= last_cut-2.5] 
        
        plt.plot((10 ** b) / (10 ** last_cut), (10 ** c) / (10 ** (slope * b + intercept)), alpha = 1, linestyle = '--', marker = 'o', label = f"m $\geq$ {m}, R $\leqslant {r}$")
        
plt.xscale('log', base = 10)
plt.yscale('log', base = 10)    
plt.xlabel("$d \ (m)$")
plt.ylabel("$\\frac{P_m(d)}{d^A\cdot10^B}$")
plt.title("Scaling picture plot")
plt.grid()
plt.legend()
plt.show()

# Dependence of the fit parameters to m and R

Previously we verified that each distribution obey a power-law regime in a defined domain of waiting times; now, we ask ourselves if these power-laws are defined by universal parameters or if those parameters depends on $m$ and $R$. To perform this investigation, we display below two heatmaps showing the values of $A$ and $B$ for different $m$ and $R$.

In [ ]:
magnitudes = np.linspace(2, 4.1, 24)  
Radii = np.linspace(1e4, 1e6, 24)     

bins   = [[] for _ in range(len(Radii))]
counts = [[] for _ in range(len(Radii))]
errors = [[] for _ in range(len(Radii))]

for j, radius in enumerate(Radii):
    for i, m in enumerate(magnitudes):
        dfmr, dd, wt = get_Pmr(df, m, radius)
        
        b, c, e = log_plot(wt)
        bins[j].append(b)
        counts[j].append(c)
        errors[j].append(e)

pearson_list = []

for i, m in enumerate(magnitudes):
    for j, radius in enumerate(Radii):
        bins[j][i], counts[j][i], errors[j][i], pearson = cut_tails(bins[j][i], counts[j][i], errors[j][i], threshold = 2)
        pearson_list.append(pearson[-1])

fits_data = []

for j, radius in enumerate(Radii):

    fit_res = statistic_plots(bins[j], counts[j], magnitudes, errors[j], 'wt', spec = f' ($R\leqslant{radius:.0f}$)', show_plots=False)
    fits_data.append([])
    
    for res, m, p in zip(fit_res, magnitudes, pearson_list):
        slope, intercept, pcov, chisquare , dof = res
        
        confidence_level = (1.- chi2.cdf(chisquare, dof))*100
        
        sigma_slope, sigma_intercept = np.sqrt(np.diag(pcov))
        fits_data[-1].append([slope, intercept, sigma_slope, sigma_intercept])
        '''        
        print(f"------ Earthquakes with m ≥ {m}, R ≤ {radius}  ------")
        print(f"Slope: {slope:.3f} ± {sigma_slope:.3f}")
        print(f"Intercept: {intercept:.3f} ± {sigma_intercept:.3f}")
        print(f"Pearson coefficient: {p:.2f}")
        print(f"Degrees of freedom: {dof}")
        #print(f"Chi2: {chisquare:.2f}")
        #print(f"Hypotesis accepted with confidence level {confidence_level:.1f}%")
        print()'''
fits_data = np.array(fits_data)

In [ ]:
data_to_plot = fits_data[:,:,0]

fig = plt.figure(figsize=(12,5))

ax1 = fig.add_subplot(1,2,1)
plot1 = ax1.contourf(data_to_plot)
ax1.set_xlabel("Magnitudes")
ax1.set_ylabel("Maximum R [m]")
ax1.set_title("Slope Contour Plot")
fig.colorbar(plot1, ax=ax1)  

x_tick_step = 4
y_tick_step = 4
ax1.set_xticks(np.arange(0, len(magnitudes), x_tick_step), ["{:.1f}".format(mag) for mag in magnitudes[::x_tick_step]])
ax1.set_yticks(np.arange(0, len(Radii), y_tick_step), ["{:.0e}".format(r) for r in Radii[::y_tick_step]])


x, y = np.meshgrid(magnitudes, Radii)
ax2 = fig.add_subplot(1,2,2, projection='3d')
scat2 = ax2.plot_surface(x, y, data_to_plot, cmap='viridis', linewidth=0, antialiased=False)
ax2.set_xlabel("Magnitudes")
ax2.set_ylabel("Maximum R [m]")
ax2.set_title("Slope 3D Plot")
fig.colorbar(scat2, ax=ax2, shrink=0.5)

plt.tight_layout()
plt.show()


In [ ]:
data_to_plot = fits_data[:,:,1]

fig = plt.figure(figsize=(12,5))

ax1 = fig.add_subplot(1,2,1)
plot1 = ax1.contourf(data_to_plot)
ax1.set_xlabel("Magnitudes")
ax1.set_ylabel("Maximum R [m]")
ax1.set_title("Slope Contour Plot")
fig.colorbar(plot1, ax=ax1)  

x_tick_step = 4
y_tick_step = 4
ax1.set_xticks(np.arange(0, len(magnitudes), x_tick_step), ["{:.1f}".format(mag) for mag in magnitudes[::x_tick_step]])
ax1.set_yticks(np.arange(0, len(Radii), y_tick_step), ["{:.0e}".format(r) for r in Radii[::y_tick_step]])


x, y = np.meshgrid(magnitudes, Radii)
ax2 = fig.add_subplot(1,2,2, projection='3d')
scat2 = ax2.plot_surface(x, y, data_to_plot, cmap='viridis', linewidth=0, antialiased=False)
ax2.set_xlabel("Magnitudes")
ax2.set_ylabel("Maximum R [m]")
ax2.set_title("Slope 3D Plot")
ax2.view_init(30,60)
fig.colorbar(scat2, ax=ax2, shrink=0.5)

plt.tight_layout()
plt.show()

From these plots we can see a dependance of the parameters with respect to $m$ and $R$. So, the power-laws are not universal, but one can try to extrapolate an empirical law through a curve fit on these maps (probably a quadratic one is a good approximation given the shape of the heatmaps).

# 6. Scaling Picture Analysis

## 6.1 Tree structure

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage

df['Fol event'] = [[] for _ in range(len(df))]

for i in df['Index']:
    if df['Prev_event'][i] > -1:
        df['Fol event'][df['Prev_event'][i]].append(i)
df

In [ ]:
#Let's select only the events with the biggest number of sons
#Let's consider only starting from 10

#voglio selezionare soltanto gli elementi che hanno un tot numero di terremoti "figli"sr
#richiediamo che len(df['Fol event']) > 10, ossia il terremoto in questione ha generato almeno 10 figli

indeces = []
sons = []


for i in range(len(df)):
    if len(df['Fol event'][i]) > 100:
        indeces.append(df['Index'][i])
        sons.append(df['Fol event'][i])

#selezioniamo gli indici in base ai terremoti che hanno avuto il maggior numero di eventi (+ di 10)

print('Number of eartquakes selected: ', len(indeces)) #stampo qui il numero di terremoti che ho selezionato con il constraint che ho imposto

In [ ]:
#Let's perform a plot to visualize the distribution of points in space
#It can be removed

list_ = [len(sons[i]) for i in range(len(indeces))]

store = []

for i in indeces:
    rand = np.random.uniform()

    color = np.random.rand(3)
    index = [i for _ in range(len(df['Fol event'][i]))]
    store.append(index)

    plt.scatter(index, df['Fol event'][i], c = [color]*len(index), cmap = 'viridis', s = 1)


plt.xlabel('Father (Index)')
plt.ylabel('Son (Index)')
plt.title('Earthquakes distribution as a function of the original earthquake')
plt.show()

**Construction of the hierarchy linkage:**

Let's perform the hierarchical/agglomerative clustering. The input y may be either a 1-D condensed distance matrix or a 2-D array of observation vectors. The linkage function is used to compute the distance matrix for the points in space. Each point will be represented by the index of the father and the index of the son, in the following way:

[[0,1], [0,2], [0,3], ..., [0, 10] , ... ]

where 0 is the original earthquake that actually causes the origin of earthquake 10. Linkage function will divide the points in clusters using as metric the centroid method: distance between different clusters will be evaluated computing the distance between the centroids. This is used to guarantee the indipendency of the events.

Let's apply the **dendrogram algorithm**: The dendrogram illustrates how each cluster is composed by drawing a U-shaped link between a non-singleton cluster and its children. The top of the U-link indicates a cluster merge. The two legs of the U-link indicate which clusters were merged. The length of the two legs of the U-link represents the distance between the child clusters. It is also the cophenetic distance between original observations in the two children clusters.

Let's see the method used:

* truncate_mode: shows only the first p clusters, while the remaining will be represented as leaves
* count_sort

*False*: nothing is done

*ascending*: the child with the minimum number of original objects in its cluster is plotted first

*descending*: the child with the maximum number of original objects in its cluster is plotted first

* show_contracted: when True the heights of non-singleton nodes contracted into a leaf node are plotted as crosses along the link connecting that leaf node. This really is only useful when truncation is used.

The output of the function is a dendrogram: on the x axis there are the labels, they can be accessed using d['ivl']. They represent the index of the earthquake that is been generated. Please notice that in most cases the label will be something like (20), because we are using the truncated mode and a single node is representing 20 earthquakes (too many data). On the y axis there will be the distance between clusters: the higher is the value of y, the biggest is the distance between different clusters.

In [ ]:
L = [[store[i][j], sons[i][j]] for i in range(len(store)) for j in range(len(store[i]))]
#L is a list containing father and sons

a = list_[0]
b = list_[1]

s = 2
i = 0
counter = 1

index = 0

while s<len(list_):
    if counter == 1 and s == 2:
        fig, axes = plt.subplots(1, 3, figsize = (20,5))

    a = b+1
    b += list_[s]

    S = np.array(L[a : b]) #divido in base ai padri
    labels = [S[j][1] for j in range(len(S))]

    z = linkage(S, method = 'centroid', optimal_ordering = False )
    #optimal_ordering = True migliora il plot finale ma il run si allunga, runnare prima della consegna


    d = dendrogram(z, truncate_mode = 'lastp', p = 15, count_sort = 'descending', show_contracted = True, leaf_rotation = 50, labels = labels, ax = axes[index] )
    #inserendo p = 5 vedo soltanto 5 labels, le restanti sono raggruppate in centroidi
    #con p posso variare il numero di labels visibili, ogni label sarà ravvicinata ad un dato terremoto
    #labels: voglio che usi come label per l'algoritmo gli index dei figli



    axes[index].set_title(f'Dendogram for earthquake {indeces[i]}')

    s += 1
    i += 1
    counter += 1
    index+= 1

    if i % 3 == 0 and s < len(list_) - 1:
        fig, axes = plt.subplots(1, 3, figsize = (20,5))
        counter = 1
        index = 0

plt.show()